# Chapter 3 — Elementary Matrices (Colab Companion)

This notebook provides **Python helpers and demos** for core topics of Chapter 3:

- The three types of **elementary matrices** (row scaling `D_i(α)`, row swap `P_ij`, row addition `E_ij(α)`)
- Implementing **row operations** via **left-multiplication** and **column operations** via **right-multiplication**
- **Inverses** of elementary matrices and their **determinants**
- **Gaussian elimination** using a running product of elementary matrices, yielding `R = E · A`
- **Factoring an invertible matrix as a product of elementary matrices**
- Bonus: Using the same machinery to compute **A^{-1}** (Gauss–Jordan)

All code uses **exact arithmetic** (rationals) via SymPy where possible.

In [ ]:
# Setup (NumPy/SymPy). In Colab these are typically available.
# If needed, you can uncomment the next line to install/upgrade:
# !pip -q install numpy sympy

import numpy as np
import sympy as sp

sp.init_printing(use_unicode=True)

def to_M(A):
    """Coerce to a SymPy Matrix with rational entries when reasonable."""
    if isinstance(A, sp.MatrixBase):
        M = A.copy()
    else:
        M = sp.Matrix(A)
    def _coerce(x):
        if isinstance(x, (int, np.integer)):
            return sp.Rational(int(x))
        try:
            return sp.nsimplify(x)
        except Exception:
            return x
    return M.applyfunc(_coerce)

## 1) Elementary matrices — constructors

We use **1-based indexing** for rows/columns to match the math notation.

In [ ]:
def D_i(n, i, a):
    """Diagonal scaling matrix: scales row i by a (1-based i)."""
    I = sp.eye(n)
    I[i-1, i-1] = to_M([[a]])[0,0]
    return I

def P_ij(n, i, j):
    """Permutation matrix swapping rows i and j (1-based)."""
    if i == j:
        return sp.eye(n)
    P = sp.eye(n)
    P.row_swap(i-1, j-1)
    return P

def E_ij(n, i, j, a):
    """Elementary matrix adding a * row j to row i (1-based, i != j)."""
    if i == j:
        raise ValueError("E_ij requires i != j")
    E = sp.eye(n)
    E[i-1, j-1] = to_M([[a]])[0,0]
    return E

## 2) Row/column operations via multiplication

- **Left-multiply** by an elementary matrix to perform the corresponding **row** operation.
- **Right-multiply** by an elementary matrix to perform the corresponding **column** operation.

In [ ]:
def left_apply(E, A):
    return to_M(E) * to_M(A)

def right_apply(A, E):
    return to_M(A) * to_M(E)

# Demo
if __name__ == "__main__":
    A = sp.Matrix([[1,2,3],[4,5,6],[7,8,9]])
    print("A=\n"); sp.pprint(A)
    print("\nSwap R1 <-> R3:")
    sp.pprint(left_apply(P_ij(3,1,3), A))
    print("\nR2 := R2 + 3*R1:")
    sp.pprint(left_apply(E_ij(3,2,1,3), A))
    print("\nScale column 3 by 2 (right-multiply by D_3(2)):")
    sp.pprint(right_apply(A, D_i(3,3,2)))

## 3) Determinants and inverses of elementary matrices

Key facts:
- `det(D_i(α)) = α`
- `det(P_ij) = -1` when `i != j`
- `det(E_ij(α)) = 1`
- Each elementary matrix is **invertible** and its inverse is of the same type.

In [ ]:
def det_elementary(M):
    return sp.Matrix(M).det()

def inv_elementary(M):
    return sp.Matrix(M).inv()

# Quick self-test
if __name__ == "__main__":
    n = 4
    D = D_i(n, 2, 7)
    P = P_ij(n, 1, 4)
    E = E_ij(n, 3, 2, -5)
    print("det(D_i(7)):", det_elementary(D))
    print("det(P_14):", det_elementary(P))
    print("det(E_32(-5)):", det_elementary(E))
    # Check inverses exist and are same type
    sp.pprint(inv_elementary(D))
    sp.pprint(inv_elementary(P))
    sp.pprint(inv_elementary(E))

## 4) Gaussian elimination with accumulated elementary matrices

We compute `R = E · A` where `E = Ek ··· E2 E1` is a product of elementary matrices.
Returned `E` is explicit (not just a single matrix product).

In [ ]:
def row_reduce_with_Es(A, pivoting=True):
    """
    Row-reduce A using elementary matrices.
    Returns (RREF, E_list, E_product) with RREF = E_product * A.
    """
    M = to_M(A)
    m, n = M.shape
    i = j = 0
    E_list = []
    def apply_and_store(E):
        nonlocal M
        E = to_M(E)
        M = E * M
        E_list.append(E)
    while i < m and j < n:
        # Choose pivot row
        pivot_row = None
        if pivoting:
            # argmax |M[r,j]| for r in [i..m-1]
            absvals = [abs(M[r,j]) for r in range(i, m)]
            if any(val != 0 for val in absvals):
                r_rel = max(range(len(absvals)), key=lambda k: absvals[k])
                pivot_row = i + r_rel
        else:
            for r in range(i, m):
                if M[r,j] != 0:
                    pivot_row = r; break
        if pivot_row is None:
            j += 1
            continue
        # Swap
        if pivot_row != i:
            E = P_ij(m, i+1, pivot_row+1)
            apply_and_store(E)
        # Scale to 1
        piv = M[i,j]
        if piv != 1:
            E = D_i(m, i+1, sp.Rational(1,1)/piv)
            apply_and_store(E)
        # Eliminate others in column j
        for r in range(m):
            if r != i and M[r,j] != 0:
                a = M[r,j]
                E = E_ij(m, r+1, i+1, -a)
                apply_and_store(E)
        i += 1; j += 1
    # Product E = Ek ... E1
    E_prod = sp.eye(m)
    for E in reversed(E_list):
        E_prod = E * E_prod
    return M, E_list, E_prod

# Demo
if __name__ == "__main__":
    A = sp.Matrix([[2,1,-1],[ -3,-1,2 ], [ -2,1,2 ]])
    R, Es, Eprod = row_reduce_with_Es(A, pivoting=True)
    print("RREF=\n"); sp.pprint(R)
    print("E·A equals R? ", (Eprod*A == R))
    print("Number of elementary steps:", len(Es))

## 5) Factor an invertible matrix as a product of elementary matrices

If `A` is invertible, the sequence of elimination steps that turns `A` into `I` yields
`E · A = I`, hence `A = E^{-1}` and **`A` is a product of elementary matrices**.
We also provide a Gauss–Jordan-based inverse.

In [ ]:
def factor_into_elementaries(A):
    """Return a list [F1, F2, ..., Fk] of elementary matrices whose product equals A."""
    A = to_M(A)
    m, n = A.shape
    if m != n:
        raise ValueError("A must be square to be invertible.")
    R, Es, Eprod = row_reduce_with_Es(A, pivoting=True)
    if R != sp.eye(m):
        raise ValueError("A is not invertible (failed to reach identity).")
    # Eprod * A = I  =>  A = (Eprod)^{-1}
    # But Eprod is itself a product of elementaries; invert by inverting each in reverse order.
    inv_factors = [e.inv() for e in Es]  # inverse of each step
    # Since R = Ek...E1 A = I, we have A = (Ek...E1)^{-1} = E1^{-1} ... Ek^{-1}
    factors = list(reversed(inv_factors))
    return factors

def inverse_via_elementaries(A):
    """Return A^{-1} using an augmented [A | I] and row operations (Gauss–Jordan)."""
    A = to_M(A)
    n = A.shape[0]
    Aug = A.row_join(sp.eye(n))
    R, Es, Eprod = row_reduce_with_Es(Aug, pivoting=True)
    left = R[:, :n]
    right = R[:, n:]
    if left != sp.eye(n):
        raise ValueError("A is singular; left block not identity.")
    return right

# Demo
if __name__ == "__main__":
    A = sp.Matrix([[1,2,1],[0,1,3],[2,0,1]])
    Fs = factor_into_elementaries(A)
    # Verify product(Fs) == A
    P = sp.eye(A.shape[0])
    for F in Fs:
        P = F * P
    print("Product equals A? ", P.equals(A))
    print("A^{-1} via elementaries:\n"); sp.pprint(inverse_via_elementaries(A))

## 6) Column operations (right-multiplication)

`A · P_ij` swaps columns `i` and `j`, `A · D_i(α)` scales column `i`, and `A · E_ij(α)` adds α times column `j` to column `i`.

In [ ]:
if __name__ == "__main__":
    A = sp.Matrix([[1,2,3],[4,5,6],[7,8,9]])
    print("Swap columns 1 and 3:")
    sp.pprint(right_apply(A, P_ij(3,1,3)))
    print("\nC2 := C2 + 4*C1:")
    sp.pprint(right_apply(A, E_ij(3,2,1,4)))
    print("\nScale column 1 by -2:")
    sp.pprint(right_apply(A, D_i(3,1,-2)))

---

### How to use this notebook

1. Run the **Setup** cell.
2. Use **`D_i`**, **`P_ij`**, **`E_ij`** to build elementary matrices.
3. Apply them via **left_apply** (rows) or **right_apply** (columns).
4. Explore **determinants/inverses** with `det_elementary` and `inv_elementary`.
5. Use **`row_reduce_with_Es`** to see elimination as a product of elementary matrices.
6. If `A` is invertible, get a **factorization** with `factor_into_elementaries(A)` and the **inverse** via `inverse_via_elementaries(A)`.